In [1]:
import pandas as pd 
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES

list_indexes = []
for dimension in DIMENSIONS:
    for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]:
        list_indexes.append([dimension, category])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension", "category"])

list_columns = []
for item in ["total", "significant", "accelerated_aging", "decelerated_aging"]:
    if item == "total":
        observations = ["total"]
    else:
        observations = ["number", "percentage"]
    for observation in observations:
        list_columns.append([item, observation])
    columns = pd.MultiIndex.from_tuples(list_columns, names=["item", "observation"])
summary = pd.DataFrame(None, index=indexes, columns=columns)
summary

item                                   total significant             \
observation                            total      number percentage   
dimension       category                                              
Abdomen         Alcohol                  NaN         NaN        NaN   
                AnthropometryBodySize    NaN         NaN        NaN   
                AnthropometryImpedance   NaN         NaN        NaN   
                ArterialStiffness        NaN         NaN        NaN   
                BloodBiochemistry        NaN         NaN        NaN   
...                                      ...         ...        ...   
set_instances23 All_Biomarkers           NaN         NaN        NaN   
                All_Phenotypes           NaN         NaN        NaN   
                All_Diseases             NaN         NaN        NaN   
                All_Environmental        NaN         NaN        NaN   
                All_Socioeconomics       NaN         NaN        NaN   

item                                   accelerated_aging             \
observation                                       number percentage   
dimension       category                                              
Abdomen         Alcohol                              NaN        NaN   
                AnthropometryBodySize                NaN        NaN   
                AnthropometryImpedance               NaN        NaN   
                ArterialStiffness                    NaN        NaN   
                BloodBiochemistry                    NaN        NaN   
...                                                  ...        ...   
set_instances23 All_Biomarkers                       NaN        NaN   
                All_Phenotypes                       NaN        NaN   
                All_Diseases                         NaN        NaN   
                All_Environmental                    NaN        NaN   
                All_Socioeconomics                   NaN        NaN   

item                                   decelerated_aging             
observation                                       number percentage  
dimension       category                                             
Abdomen         Alcohol                              NaN        NaN  
                AnthropometryBodySize                NaN        NaN  
                AnthropometryImpedance               NaN        NaN  
                ArterialStiffness                    NaN        NaN  
                BloodBiochemistry                    NaN        NaN  
...                                                  ...        ...  
set_instances23 All_Biomarkers                       NaN        NaN  
                All_Phenotypes                       NaN        NaN  
                All_Diseases                         NaN        NaN  
                All_Environmental                    NaN        NaN  
                All_Socioeconomics                   NaN        NaN  

[2970 rows x 7 columns]

In [2]:
dimension = DIMENSIONS[0]

correlations_dimension = pd.read_feather(f"../../data/xwas/univariate_results/linear_correlations_{dimension}.feather", columns=["category", "p_value", "correlation"])

for group in correlations_dimension.groupby(by=["category"]):
    category = group[0]
    summary.loc[(dimension, category), ("total", "total")] = group[1].shape[0]
    summary.loc[(dimension, category), ("significant", "number")] = (group[1]["p_value"] < 0.05 / group[1].shape[0]).sum()
    summary.loc[(dimension, category), ("accelerated_aging", "number")] = ((group[1]["p_value"] < 0.05 / group[1].shape[0]) & (group[1]["correlation"] < 0)).sum()
    summary.loc[(dimension, category), ("decelerated_aging", "number")] = ((group[1]["p_value"] < 0.05 / group[1].shape[0]) & (group[1]["correlation"] > 0)).sum()

In [3]:
for item in ["significant", "accelerated_aging", "decelerated_aging"]:
    summary.loc[dimension, (item, "percentage")] = (summary.loc[dimension, (item, "number")] / summary.loc[dimension, ("total", "total")]).values

In [4]:
for main_category, categories in MAIN_CATEGORIES_TO_CATEGORIES.items():
    group = correlations_dimension.set_index("category").loc[categories]
    
    summary.loc[(dimension, f"All_{main_category}"), ("total", "total")] = group.shape[0]
    summary.loc[(dimension, f"All_{main_category}"), ("significant", "number")] = (group["p_value"] < 0.05 / group.shape[0]).sum()
    summary.loc[(dimension, f"All_{main_category}"), ("accelerated_aging", "number")] = ((group["p_value"] < 0.05 / group.shape[0]) & (group["correlation"] < 0)).sum()
    summary.loc[(dimension, f"All_{main_category}"), ("decelerated_aging", "number")] = ((group["p_value"] < 0.05 / group.shape[0]) & (group["correlation"] > 0)).sum()

In [5]:
for item in ["significant", "accelerated_aging", "decelerated_aging"]:
    summary.loc[dimension, (item, "percentage")] = (summary.loc[dimension, (item, "number")] / summary.loc[dimension, ("total", "total")]).values

In [9]:
summary

item                                   total significant             \
observation                            total      number percentage   
dimension       category                                              
Abdomen         Alcohol                   29           7   0.241379   
                AnthropometryBodySize      8           6       0.75   
                AnthropometryImpedance     5           5        1.0   
                ArterialStiffness          8           2       0.25   
                BloodBiochemistry         28          14        0.5   
...                                      ...         ...        ...   
set_instances23 All_Biomarkers           NaN         NaN        NaN   
                All_Phenotypes           NaN         NaN        NaN   
                All_Diseases             NaN         NaN        NaN   
                All_Environmental        NaN         NaN        NaN   
                All_Socioeconomics       NaN         NaN        NaN   

item                                   accelerated_aging             \
observation                                       number percentage   
dimension       category                                              
Abdomen         Alcohol                                5   0.172414   
                AnthropometryBodySize                  1      0.125   
                AnthropometryImpedance                 5        1.0   
                ArterialStiffness                      2       0.25   
                BloodBiochemistry                      8   0.285714   
...                                                  ...        ...   
set_instances23 All_Biomarkers                       NaN        NaN   
                All_Phenotypes                       NaN        NaN   
                All_Diseases                         NaN        NaN   
                All_Environmental                    NaN        NaN   
                All_Socioeconomics                   NaN        NaN   

item                                   decelerated_aging             
observation                                       number percentage  
dimension       category                                             
Abdomen         Alcohol                                2   0.068966  
                AnthropometryBodySize                  5      0.625  
                AnthropometryImpedance                 0        0.0  
                ArterialStiffness                      0        0.0  
                BloodBiochemistry                      6   0.214286  
...                                                  ...        ...  
set_instances23 All_Biomarkers                       NaN        NaN  
                All_Phenotypes                       NaN        NaN  
                All_Diseases                         NaN        NaN  
                All_Environmental                    NaN        NaN  
                All_Socioeconomics                   NaN        NaN  

[2970 rows x 7 columns]

In [7]:
# summary.columns = map(str, summary.columns.tolist())
# summary.reset_index().to_feather("../../data/xwas/univariate_results/summary.feather")

In [8]:
summary = pd.read_feather("../../data/xwas/univariate_results/summary.feather").set_index(["dimension", "category"])
summary.columns = pd.MultiIndex.from_tuples(
    list(map(eval, summary.columns.tolist())), names=["category", "variable"]
)
summary

category                               total significant             \
variable                               total      number percentage   
dimension       category                                              
Abdomen         Alcohol                   29           7   0.241379   
                AnthropometryBodySize      8           6   0.750000   
                AnthropometryImpedance     5           5   1.000000   
                ArterialStiffness          8           2   0.250000   
                BloodBiochemistry         28          14   0.500000   
...                                      ...         ...        ...   
set_instances23 medical_diagnoses_V       85           0   0.000000   
                medical_diagnoses_W       81           0   0.000000   
                medical_diagnoses_X       90           4   0.044444   
                medical_diagnoses_Y       86           0   0.000000   
                medical_diagnoses_Z       84           1   0.011905   

category                               accelerated_aging             \
variable                                          number percentage   
dimension       category                                              
Abdomen         Alcohol                                5   0.172414   
                AnthropometryBodySize                  1   0.125000   
                AnthropometryImpedance                 5   1.000000   
                ArterialStiffness                      2   0.250000   
                BloodBiochemistry                      8   0.285714   
...                                                  ...        ...   
set_instances23 medical_diagnoses_V                    0   0.000000   
                medical_diagnoses_W                    0   0.000000   
                medical_diagnoses_X                    4   0.044444   
                medical_diagnoses_Y                    0   0.000000   
                medical_diagnoses_Z                    1   0.011905   

category                               decelerated_aging             
variable                                          number percentage  
dimension       category                                             
Abdomen         Alcohol                                2   0.068966  
                AnthropometryBodySize                  5   0.625000  
                AnthropometryImpedance                 0   0.000000  
                ArterialStiffness                      0   0.000000  
                BloodBiochemistry                      6   0.214286  
...                                                  ...        ...  
set_instances23 medical_diagnoses_V                    0   0.000000  
                medical_diagnoses_W                    0   0.000000  
                medical_diagnoses_X                    0   0.000000  
                medical_diagnoses_Y                    0   0.000000  
                medical_diagnoses_Z                    0   0.000000  

[2772 rows x 7 columns]